# Common event display

Edit the config in the second cell in step 1. Then hit `shift-enter` in each cell to run them.  
Actual event display occurs on the last cell...but all previous celle need to be run previously

In [2]:
import uproot
import pandas as pd
from tqdm.auto import tqdm

%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

1. Edit the following config cell (run_number, files paths, position adjustement, etc.)

In [3]:
save_display = False # Auto save each event display, Saving is actually quite long. Set to False to just look at the events
run_number = 744318
shift_window = 10 # In clock
bcid_shift = 2488

hcal_pads = 96
hcal_padsize = 10.408
hcal_layersize = 28
hcal_layers = 37
hcal_side = hcal_pads * hcal_padsize
hcal_depth = hcal_layers * hcal_layersize

shift_x = 225 # distance from the left edge of ECAL and left edge of HCAL
shift_y = 377 # distance from the bottom edge of ECAL and bottom edge of HCAL

ecal_pixels = 32
ecal_pixelsize = 5.5
ecal_layersize = 15
ecal_slabs = 10
ecal_side = ecal_pixels * ecal_pixelsize
ecal_depth = ecal_slabs * ecal_layersize
ecal_shiftx = shift_x + ecal_side/2 + ecal_pixelsize/2
ecal_shifty = shift_y + ecal_side/2 + ecal_pixelsize/2

# Canvas size, should not need to update
w, h, dpi = 800, 600, 96 

# Read data directly from eos via xroot, you need to be identified with a kerberos5 token to use this feature
use_xroot = True
xroot_url = 'root://eosuser.cern.ch/'

hcal_file_name = '/eos/user/a/apingaul/CALICE/Data/SPS_09_2018/Trivent/DHCAL_Trivent_Ecal_{}.root'.format(run_number)
hcal_tree_name = 'sdhcal'
hcal_branches = ['EvtNum', 'HitX', 'HitY', 'HitZ', 'HitThresh']

# ecal_file_name = '/eos/project/s/siw-ecal/TB2018-09/Common/ECAL/Muon_200GeV/{}__build.root'.format(run_number)
ecal_file_name = '/eos/project/s/siw-ecal/TB2018-09/Common/ECAL/PiPlus_70GeV/{}__build.root'.format(run_number)
ecal_tree_name = 'ecal'
ecal_branches = ['event', 'hit_x', 'hit_y', 'hit_z','nhit_slab']


common_file_name = '/eos/user/a/apingaul/CALICE/Data/SPS_09_2018/Trivent/Common_{}_bcid_corrected.root'.format(run_number)
common_tree_name = 'common'

if use_xroot:
    hcal_file_name = xroot_url + hcal_file_name
    ecal_file_name = xroot_url + ecal_file_name
    common_file_name = xroot_url + common_file_name

2. Open the files (depending on the number of events in ecal/hcal this might take up to a couple minutes...)

In [22]:
df_common = uproot.open(common_file_name)[common_tree_name].pandas.df().query('delta_revbcid_corrected>={} and delta_revbcid_corrected<={}'.format(bcid_shift-shift_window,bcid_shift+shift_window))
display('common ok')
# df_hcal = uproot.open(hcal_file_name)[hcal_tree_name].pandas.df(hcal_branches)
# display('hcal ok')
# df_ecal = uproot.open(ecal_file_name)[ecal_tree_name].pandas.df(ecal_branches)
# display('ecal ok')

'common ok'

3. Drawing functions definition, nothing to do just run it (with `shift-enter`)

In [23]:
shift_zhcal = df_hcal['HitZ'].min()
max_zhcal = df_hcal['HitZ'].max()

def draw_calo_silouhette(ax, draw_ecal=True, draw_hcal=True):
    if draw_ecal:
        draw_options = {
            'linewidth': 1,
            'linestyle': '--',
            'edgecolor': 'orange',
            'fill': False
        }
        ax[0].add_patch(Rectangle((shift_x, shift_y), ecal_side,  ecal_side, **draw_options))
        ax[1].add_patch(Rectangle((0, shift_x),       ecal_depth, ecal_side, **draw_options))
        ax[2].add_patch(Rectangle((0, shift_y),       ecal_depth, ecal_side, **draw_options))

    
    if draw_hcal:
        draw_options = {
            'linewidth': 1,
            'linestyle': '--',
            'edgecolor': 'black',
            'fill': False
        }
        ax[0].add_patch(Rectangle((0, 0),           hcal_side,  hcal_side, **draw_options))
        ax[1].add_patch(Rectangle((shift_zhcal, 0), hcal_depth, hcal_side, **draw_options))
        ax[2].add_patch(Rectangle((shift_zhcal, 0), hcal_depth, hcal_side, **draw_options))

# ------------------------------------------------------------------------ 
# ------------------------------------------------------------------------ 
def plot_common(iEntry, save_display, canvas_options=None, marker_size=2, draw_ecal=True, draw_hcal=True):
    plt.close('all') # Clean the list of figures, handy when calling multiple times this cell
    with plt.xkcd(.5, 100, 2):
        fig_xyz, ax_xyz = plt.subplots(**canvas_options)
        ax_xyz[0].set(xlabel='X (mm)', ylabel='Y (mm)', title='Front view')
        ax_xyz[1].set(xlabel='Z (mm)', ylabel='X (mm)', title='Top View')
        ax_xyz[2].set(xlabel='Z (mm)', ylabel='Y (mm)', title='Side View')

        axis_overflow=50
        ax_xyz[0].set_xlim(-axis_overflow, hcal_side + axis_overflow)
        ax_xyz[0].set_ylim(-axis_overflow, hcal_side + axis_overflow )

        ax_xyz[1].set_xlim(-axis_overflow, shift_zhcal + hcal_depth + axis_overflow)
        ax_xyz[1].set_ylim(-axis_overflow, hcal_side + axis_overflow)

        ax_xyz[2].set_xlim(-axis_overflow, shift_zhcal + hcal_depth + axis_overflow)
        ax_xyz[2].set_ylim(-axis_overflow, hcal_side + axis_overflow)
    
    df_entry = df_common.iloc[iEntry]
    trigger = df_entry['ecal_spill']
    ecal_evt = df_entry['ecal_event']
    hcal_evt = df_entry['sdhcal_EvtNum']
    delta_bcid = df_entry['delta_revbcid_corrected']
    print('Trigger/spill: {}'.format(trigger))
    fig_xyz.suptitle('Run={} - trig/spill={} - ecal_evt={} - sdhcal_evt={} - $\Delta$bcid={}'.format(run_number, trigger, ecal_evt, hcal_evt, delta_bcid))
    print('\t Events: sdhcal: {} - ecal: {} - Delta_bcid: {}'.format(hcal_evt, ecal_evt, delta_bcid))

    df_hcal_evt = df_hcal.query('EvtNum=={}'.format(hcal_evt))
    df_ecal_evt = df_ecal.query('event=={}'.format(ecal_evt))
    color = ['g', 'b', 'r']
    for thresh in range(1, 4):
        df_thresh = df_hcal_evt.query('HitThresh=={}'.format(thresh))
        ax_xyz[0].scatter(df_thresh['HitX'], df_thresh['HitY'], c=color[thresh-1], s=marker_size)
        ax_xyz[1].scatter(df_thresh['HitZ'], df_thresh['HitX'], c=color[thresh-1], s=marker_size)
        ax_xyz[2].scatter(df_thresh['HitZ'], df_thresh['HitY'], c=color[thresh-1], label='sdhcal_thresh{}'.format(thresh), s=marker_size)

    # Ecal, (0, 0) of the ecal hits at the center of the ecal -> hit need to be repositionned on the common frame, 
    ax_xyz[0].scatter(df_ecal_evt['hit_x'] + ecal_shiftx,    df_ecal_evt['hit_y'] + ecal_shifty, c='orange', s=marker_size)
    ax_xyz[1].scatter(df_ecal_evt['hit_z'] * ecal_layersize, df_ecal_evt['hit_x'] + ecal_shiftx, c='orange', s=marker_size)
    ax_xyz[2].scatter(df_ecal_evt['hit_z'] * ecal_layersize, df_ecal_evt['hit_y'] + ecal_shifty, c='orange', label='ecal', s=marker_size)

    # Draw ecal silouhette
    draw_calo_silouhette(ax_xyz, draw_ecal, draw_hcal)

    # Show the legend
    with plt.xkcd(.5, 100, 2):
        ax_xyz[2].legend(loc='upper right', fontsize='x-small')
        
    fig_xyz.subplots_adjust(top=0.8)
    
    if save_display:
        fig_xyz.savefig('./Figs/Event_display_{}_trig{}_ecal{}_sdhcal{}.png'.format(run_number, trigger, ecal_evt, hcal_evt), transparent=False, dpi=96, bbox_inches="tight")


# Make some widget to interact with
slider_event = widgets.IntSlider(value=0, max=len(df_common), description='Entry')
button_next = widgets.Button(description='Next')
button_prev = widgets.Button(description='Prev')
button_save = widgets.Button(description='Save')

def next_event(b):
    slider_event.value += 1
    plot_common(slider_event.value)

def prev_event(b):
    slider_event.value -= 1
    plot_common(slider_event.value)

def save_event(b):
    plot_common(slider_event.value, True)

button_next.on_click(next_event)
button_prev.on_click(prev_event)
button_save.on_click(save_event)
hbox = widgets.HBox([button_prev, button_next, button_save])

slider_marker = widgets.IntSlider(value=2, max=10, description='Marker size')
button_ecal = widgets.Checkbox(description='Show ecal', value=True)
button_hcal = widgets.Checkbox(description='Show hcal', value=True)

canvas_options = {
    'nrows' : 1,
    'ncols': 3,
    'dpi' : dpi,
}
canvas_options['figsize'] = canvas_options['ncols']*w/dpi, canvas_options['nrows']*h/dpi
plts = widgets.interactive(plot_common, iEntry=slider_event, save_display=fixed(save_display), canvas_options=fixed(canvas_options), marker_size=slider_marker, show_ecal=button_ecal, show_hcal=button_hcal)

4. Finally draw the event display

In [24]:
display(plts, hbox)

interactive(children=(IntSlider(value=0, description='Entry', max=310), IntSlider(value=2, description='Marker…